In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-156484
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-156484


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name = "project1Cluster"
try:
    target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Same cluster name compute exists. Reusing the same")
except: 
    print("Compute with same cluster name doesn't exist and hence creating NEW")
    config_compute = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes = 5)
    target = ComputeTarget.create(ws, cluster_name, config_compute)

target.wait_for_completion(show_output=True)
print(target.get_status().serialize())


Compute with same cluster name doesn't exist and hence creating NEW
InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2021-08-31T19:40:20.862000+00:00', 'errors': None, 'creationTime': '2021-08-31T19:40:20.499777+00:00', 'modifiedTime': '2021-08-31T19:40:46.094290+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 5, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:
print(2+3)

5


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C' : choice(0.001,0.01,0.1,1,10,50,100,150,200,300,400, 500,1000),
    '--max_iter' : choice(50,100,150,200,300)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
script_folder = './training'
os.makedirs(script_folder, exist_ok=True)
import shutil
shutil.copy('./train.py', script_folder)
est = SKLearn(
    source_directory= script_folder,
    compute_target=target,
    entry_script="train.py"
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name='ROC_Weighted',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=policy,
    estimator=est,
    max_total_runs=15,
    max_concurrent_runs=3

)

In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
run_hyp = exp.submit(hyperdrive_config)
RunDetails(run_hyp).show()

run_hyp.wait_for_completion(show_output = True)
#assert(run_hyp.get_status() =="Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e354b747-f90c-4124-a95b-40a1e6c38010
Web View: https://ml.azure.com/runs/HD_e354b747-f90c-4124-a95b-40a1e6c38010?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-156484/workspaces/quick-starts-ws-156484&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-08-31T20:10:41.725340][API][INFO]Experiment created<END>\n""<START>[2021-08-31T20:10:42.174678][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space<END>\n""<START>[2021-08-31T20:10:42.597737][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_e354b747-f90c-4124-a95b-40a1e6c38010
Web View: https://ml.azure.com/runs/HD_e354b747-f90c-4124-a95b-40a1e6c38010?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-156484/workspaces/quick-starts-ws-156484&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_e354b747-f90c-4124-a95b-40a1e6c38010',
 'target': 'project1Cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-31T20:10:41.361805Z',
 'endTimeUtc': '2021-08-31T20:24:21.839969Z',
 'properties': {'primary_metric_config': '{"name": "ROC_Weighted", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e078d434-f0bc-4e29-af6f-d23596346989',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0',
  'score': '0.933192362056231',
  'best_child_run_id': 'HD_e354b747-f90c-4124-a95b-40a1e6c38010_7',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg156484.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_e354b747-f90c-4124-a95b-40a1e6c38010/azureml-logs/hyperdrive.txt?sv=2019-07-07

KeyError: 'log_files'

In [10]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run_hyp.get_best_run_by_primary_metric()
print("Best Run File names:")
print(best_run.get_file_names())
print("Best Run Details:")
print(best_run.get_details())
print("Best Run Metrics")
print(best_run.get_metrics())

Best Run File names:
['azureml-logs/55_azureml-execution-tvmps_367d311a1a7c18e3137e38ff48a9120aa9e5bf5b49230b4148b0133b3c6eb9b8_d.txt', 'azureml-logs/65_job_prep-tvmps_367d311a1a7c18e3137e38ff48a9120aa9e5bf5b49230b4148b0133b3c6eb9b8_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_367d311a1a7c18e3137e38ff48a9120aa9e5bf5b49230b4148b0133b3c6eb9b8_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/110_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']
Best Run Details:
{'runId': 'HD_e354b747-f90c-4124-a95b-40a1e6c38010_7', 'target': 'project1Cluster', 'status': 'Completed', 'startTimeUtc': '2021-08-31T20:16:54.840231Z', 'endTimeUtc': '2021-08-31T20:18:18.93325Z', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': 'e078d434-f0bc-4e29-af6f-d23596346989', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azurem

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [12]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [13]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=4)

In [14]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output = False)
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_b8a1f7b2-b6de-40eb-b7c0-7d90d727ded4,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [15]:
# Retrieve and save your best automl model.

### YOUR CODE##
run_automl = automl_run.get_best_child()
run_automl.get_details()

{'runId': 'AutoML_b8a1f7b2-b6de-40eb-b7c0-7d90d727ded4_30',
 'status': 'Completed',
 'startTimeUtc': '2021-08-31T20:56:05.058256Z',
 'endTimeUtc': '2021-08-31T20:56:58.948739Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-project\',\'compute_target\':\'local\',\'subscription_id\':\'3d1a56d2-7c81-4118-9790-f85d1acf0c77\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_b8a1f7b2-b6de-40eb-b7c0-7d90d727ded4_30","experiment_name":null,"workspace_name":"quick-starts-ws-156484","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group_name":"aml